# MNIST Classification with DNN

In [0]:
import tensorflow as tf

In [0]:
tf.reset_default_graph()

In [0]:
#Set random seed for reproducible results
tf.set_random_seed(42)

## Step 1 : Collect Data

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Get Training and Test Data

In [0]:
trainX = mnist.train.images
trainY = mnist.train.labels

In [0]:
testX = mnist.test.images
testY = mnist.test.labels

Lets define some parameters

In [0]:
#Directory to save logs and graph, change it as needed
logs_path=''

#Learning rate
learning_rate = 0.03

#Number of input features - MNIST has 784 features
n_features = trainX.shape[1]

#Number of possible output classess - 10 for MNIST
n_classes = trainY.shape[1]

#Model name for storage
model_name = 'mnist_dnn.ckpt'

#How many examples to feed for training at one time
batch_size = 100

#How many times all the data to be shown
training_epochs = 60

#Define number of neurons in 4 hidden layers
K = 200
L = 100
M = 60
N = 30

# Build the Graph

Input placeholders

In [0]:
with tf.name_scope('input'):
    
    # None -> batch size can be any size, with n_features
    x = tf.placeholder(tf.float32, shape=[None, n_features], name="x-input") 
    
    # target n_classes output classes
    y_ = tf.placeholder(tf.float32, shape=[None, n_classes], name="y-input")

Layer 1

In [0]:
with tf.name_scope('layer_1'):
    W1 = tf.Variable(tf.truncated_normal([n_features, K], stddev=0.1))
    b1 = tf.Variable(tf.zeros([K]))
    Y1 = tf.nn.sigmoid(tf.add(tf.matmul(x,W1),b1))

Layer 2

In [0]:
with tf.name_scope('layer_2'):
    W2 = tf.Variable(tf.truncated_normal([K, L], stddev=0.1))
    b2 = tf.Variable(tf.zeros([L]))
    Y2 = tf.nn.sigmoid(tf.add(tf.matmul(Y1,W2),b2))

Layer 3

In [0]:
with tf.name_scope('layer_3'):
    W3 = tf.Variable(tf.truncated_normal([L, M], stddev=0.1))
    b3 = tf.Variable(tf.zeros([M]))
    Y3 = tf.nn.sigmoid(tf.add(tf.matmul(Y2,W3),b3))

Layer 4

In [0]:
with tf.name_scope('layer_4'):
    W4 = tf.Variable(tf.truncated_normal([M, N], stddev=0.1))
    b4 = tf.Variable(tf.zeros([N]))
    Y4 = tf.nn.sigmoid(tf.add(tf.matmul(Y3,W4),b4))

Output Layer   

In [0]:
with tf.name_scope("Output"):
    W5 = tf.Variable(tf.truncated_normal([N,n_classes], stddev=0.1))
    b5 = tf.Variable(tf.zeros([n_classes]))
    
    #To use more stable version of cross_entropy loss in tensorflow
    Ylogits = tf.matmul(Y4,W5) + b5
    
    #Actual prediction
    y = tf.nn.softmax(tf.matmul(Y4,W5) + b5)

Loss

In [0]:
with tf.name_scope('Loss'):
    
    #More stable version of cross_entroy    
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=Ylogits, labels=y_)
    cross_entropy = tf.reduce_mean(cross_entropy)*100
    
    #Earlier cross_entropy implementation
    #cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

GradientDescent Optimizer

In [0]:
with tf.name_scope('train'):        
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)    

Model Accuracy

In [0]:
with tf.name_scope('Accuracy'):
    
    #Get the number with highest probability
    prediction = tf.argmax(y,1,name="Predict")
    
    #Compare prediction with actual
    correct_prediction = tf.equal(prediction, tf.argmax(y_,1))
    
    #Calculate mean accuracy across all data examples
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),
                              name="accuracy")

Loss and Accuracy Logging

In [0]:
# Training loss and accuracy
training_loss = tf.summary.scalar("training_loss", cross_entropy)
training_accuracy = tf.summary.scalar("training_accuracy", accuracy)

#Test loss and accuracy
test_loss = tf.summary.scalar("test_loss", cross_entropy)
test_accuracy = tf.summary.scalar("test_accuracy", accuracy)

# create log writer object
writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

# Execute the Graph

In [0]:
#Start Graph execution
with tf.Session() as sess:
    
    # variables need to be initialized before we can use them
    sess.run(tf.global_variables_initializer())
   
    # perform training cycles
    for epoch in range(training_epochs):
        
        # number of batches in one epoch
        batch_count = int(trainX.shape[0]/batch_size)
        
        for i in range(batch_count):
            batch_x  = trainX[i*batch_size:i*batch_size+batch_size]
            batch_y  = trainY[i*batch_size:i*batch_size+batch_size]

            # perform the operations we defined earlier on batch
            _,acc,loss = sess.run([train_op, training_accuracy,training_loss], 
                                  feed_dict={x: batch_x, y_: batch_y})
            
            #log training accuracy and loss
            writer.add_summary(acc, epoch * batch_count + i)
            writer.add_summary(loss, epoch * batch_count + i)    
                       
        #Test loss and accuracy
        #Please note we are giving test data
        acc,loss = sess.run([test_accuracy,test_loss],
                                   feed_dict={x: testX, y_: testY})
        
        writer.add_summary(acc, epoch * batch_count + i)
        writer.add_summary(loss, epoch * batch_count + i)
        
        if epoch % 5 == 0: 
            print ("Epoch: ", epoch)
            print ("Test Accuracy: ", accuracy.eval(feed_dict={x: testX, 
                                                               y_: testY}))               
    
    
    #Create a Saver to save the graph
    saver = tf.train.Saver()
    saver.save(sess, logs_path + '/' + model_name)